FASE 3: Extracción de Detalles de Actores con Selenium

In [7]:
# Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [19]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException  #Este importé para manejar más errores.
from time import sleep
import os
import signal
import sys

Del csv con todas nuestras peliculas, nos quedamos solo con la columna de los ids (la comprobamos):

In [2]:
df_pelis = pd.read_csv("peliculas_api.csv")

In [3]:
df_pelis.keys()

Index(['Unnamed: 0', 'Tipo', 'Nombre', 'Año de Estreno', 'Mes de Estreno',
       'ID de la Película', 'Genero'],
      dtype='object')

In [4]:
id_IMDB= df_pelis["ID de la Película"]
id_IMDB[:5]

0    tt0146592
1    tt0154039
2    tt0162942
3    tt0230212
4    tt0326965
Name: ID de la Película, dtype: object

In [5]:
id_IMDB[0]


'tt0146592'

Comenzamos con SELENIUM!Por ahora, solo con una pelicula:

In [31]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt0448115/")

sleep(3)
#aceptamos cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)
#ampliamos pantalla
driver.maximize_window()

#hacemos scroll hasta lxs actores y actrices y creamos dicc donde meteremos la info.
driver.execute_script("window.scrollTo(0,2000)")
sleep(2)
dicc_actors= {"Nombre actr" : [], 
              "conocide_por": [],
              "año_nacimiento":[],
              "profesión": [],
              "premios": []}

#cogemos el link de la primera persona actor/actriz de la peli, y creamos diccionario para guardar resultados.
for i in range (1,11):
    driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.gWwKlt > a").click()

    sleep(3)


    #Extraemos el nombre y apendeamos al diccionario.
    nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text
    dicc_actors["Nombre actr"].append(nombre_act)

    sleep(5)

    #Extraemos las pelis por las que se le conoce y apendeamos al diccionario.
    conocide_por= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
    dicc_actors["conocide_por"].append(conocide_por)

    sleep(2)

    conocide_por2=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
    dicc_actors["conocide_por"].append(conocide_por2)

    #Extraemos la fecha de nacimiento y apendeamos al diccionario.
    sleep(3)
    fecha_nacimiento= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-11.eyxQlE > section > aside > div > span:nth-child(2)").text
    dicc_actors["año_nacimiento"].append(fecha_nacimiento)

    #Extraemos la información sobre lo que hace y apendeamos al diccionario como "profesión".
    sleep(2)
    driver.execute_script("window.scrollTo(0,4000)")

    sleep(2)
    profesion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > ul").text
    profesion_limpia = profesion.replace("\n", ", ")
    dicc_actors["profesión"].append(profesion_limpia)


    sleep(2)
    #por último, cogemos los premios, si los tiene y los apendeamos al diccionario.
    try:
        premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text   #"css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(8) > div > ul > li > div > ul > li > span").text
        dicc_actors["premios"].append(premios)

    except NoSuchElementException:
        print("No tiene premios ni nominaciones.")

    driver.back()



ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (727, 2111)
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7DE171502+60802]
	(No symbol) [0x00007FF7DE0EAC02]
	(No symbol) [0x00007FF7DDFA7CE4]
	(No symbol) [0x00007FF7DDFFE877]
	(No symbol) [0x00007FF7DDFFC344]
	(No symbol) [0x00007FF7DDFF980A]
	(No symbol) [0x00007FF7DDFF89E2]
	(No symbol) [0x00007FF7DDFEAE8B]
	(No symbol) [0x00007FF7DE01AB7A]
	(No symbol) [0x00007FF7DDFEA7C6]
	(No symbol) [0x00007FF7DE01AD90]
	(No symbol) [0x00007FF7DE03A224]
	(No symbol) [0x00007FF7DE01A923]
	(No symbol) [0x00007FF7DDFE8FEC]
	(No symbol) [0x00007FF7DDFE9C21]
	GetHandleVerifier [0x00007FF7DE47411D+3217821]
	GetHandleVerifier [0x00007FF7DE4B60B7+3488055]
	GetHandleVerifier [0x00007FF7DE4AF03F+3459263]
	GetHandleVerifier [0x00007FF7DE22B846+823494]
	(No symbol) [0x00007FF7DE0F5F9F]
	(No symbol) [0x00007FF7DE0F0EC4]
	(No symbol) [0x00007FF7DE0F1052]
	(No symbol) [0x0

In [32]:
dicc_actors

{'Nombre actr': ['Zachary Levi'],
 'conocide_por': ['¡Shazam!', 'Chuck'],
 'año_nacimiento': ['29 de septiembre de 1980'],
 'profesión': ['Reparto, Producción, Dirección'],
 'premios': ['3 premios y 23 nominaciones']}

In [ ]:
#CSS
# #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.gWwKlt > a"
# #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2)
# #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(3)
# #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page2.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(10)

Creamos una lista vacía para los primeros 10 actores/actrices (link desde la página principal). donde solo cambia el numero, {i}

In [22]:
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/title/tt0448115/")

sleep(3)

driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(2)

driver.maximize_window()

driver.execute_script("window.scrollTo(0,2000)")
sleep(2)

dicc_actors = {"nombre_act": [], "conocide_por": []}

#me voy al primer actor/actriz
#driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.sc-bfec09a1-7.gWwKlt > a").click()
#sleep(3)

lista_url=[]
for i in range (1,11):
    lista_url.append(driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})").text)

driver.close()


Extraemos los 10 primeros nombres de los actores y actrices de la película.

In [36]:
lista_url
nombres_10 = [nombre.split('\n')[0] for nombre in lista_url]
nombres_10


['Zachary Levi',
 'Mark Strong',
 'Asher Angel',
 'Jack Dylan Grazer',
 'Adam Brody',
 'Djimon Hounsou',
 'Faithe Herman',
 'Meagan Good',
 'Grace Caroline Currey',
 'Michelle Borth']

In [34]:
nombres_10.append(dicc_actors["nombre_act"])

#Creo que hay que crear otro bucle for para que appendee

In [ ]:
#COMPROBANDO css selectores- actores
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1)"
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2)"
#"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(3)"
#num10
##__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page2.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(10)

In [22]:
####parte añadida por lydia, nada claro,lo dejo de momento por si acaso pero creo que no va a servir..
# iteramos sobre la lista de elementos
for elemento in conocide_por:
    # Acceder al texto dentro de cada elemento y agregarlo al diccionario
    texto_elemento = elemento.text
    dicc_actors["conocide_por"].append(texto_elemento)

# Imprimir el diccionario con los actores
print(dicc_actors)



titulos = []

# Iteramos sobre la lista de elementos
for elemento in conocide_por:
    # Acceder al texto dentro de cada elemento y agregarlo a la lista de títulos
    titulo_elemento = elemento.text
    titulos.append(titulo_elemento)

# Imprimir la lista de títulos
print(titulos)








{'nombre_act': [], 'conocide_por': [[<selenium.webdriver.remote.webelement.WebElement (session="2233b7d8995ac68a79291a4b3f6aac02", element="f.56DF74BB773B3C8ADF456FFECAE83645.d.8DE69033094D88C6D477C1A3E87CFC28.e.186")>], '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de TV\nChuck Bartowski\n2007–2012 • 91 episodios\nEnredados\n7,7\nFlynn Rider(voz)\n2010\nThor: El mundo oscuro\n6,8\nFandral\n2013', '¡Shazam!\n7,0\nShazam\n2019\nChuck\n8,2\nSerie de 

In [ ]:
for pelicula in web_peli:
    try:
        url_imdb= f"url{id}"
    
    except:
        pass

In [ ]:
#para buscar premios
driver.find_element("link text", "Premios").click()